In [6]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna  #载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


##optuna######
###耗费时间的过程。。。。。。。。。。。。。。。
# Define a basic convolutional layer
class RayLU(nn.Module):
    def __init__(self, trial, seqFlag):
        super(RayLU, self).__init__()
        self.sigma = nn.Parameter(torch.tensor(trial.suggest_float(f'sigma_{seqFlag}', 0.1, 5.0)))

    def forward(self, x):
        return torch.where(x >= 0, x, x * torch.exp(-x**2 / (2 * self.sigma**2)))



def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)


# 定义ResNet模型
class ResNet(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = RayLU(trial, 1)  # 自定义激活函数层

        # Layer 1
        self.layer1_conv1 = conv3x3(16, 16)
        self.layer1_bn1 = nn.BatchNorm2d(16)
        self.layer1_conv2 = conv3x3(16, 16)
        self.layer1_bn2 = nn.BatchNorm2d(16)
        self.layer1_extra_conv1 = conv3x3(16, 16)
        self.layer1_extra_bn1 = nn.BatchNorm2d(16)
        self.layer1_extra_conv2 = conv3x3(16, 16)
        self.layer1_extra_bn2 = nn.BatchNorm2d(16)
        self.gelub11 = RayLU(trial, 2)
        self.gelub12 = RayLU(trial, 3)
        self.gelub13 = RayLU(trial, 4)
        self.gelub14 = RayLU(trial, 5)

        # Layer 2
        self.layer2_conv1 = conv3x3(16, 32, stride=2)
        self.layer2_bn1 = nn.BatchNorm2d(32)
        self.layer2_conv2 = conv3x3(32, 32)
        self.layer2_bn2 = nn.BatchNorm2d(32)
        self.layer2_extra_conv1 = conv3x3(16, 32, stride=2)  # 调整residual的通道数
        self.layer2_extra_bn1 = nn.BatchNorm2d(32)
        self.layer2_extra_conv2 = conv3x3(32, 32)
        self.layer2_extra_bn2 = nn.BatchNorm2d(32)
        self.gelub21 = RayLU(trial, 6)
        self.gelub22 = RayLU(trial, 7)
        self.gelub23 = RayLU(trial, 8)
        self.gelub24 = RayLU(trial, 9)
        self.layer2_downsample = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )

        # Layer 3
        self.layer3_conv1 = conv3x3(32, 64, stride=2)
        self.layer3_bn1 = nn.BatchNorm2d(64)
        self.layer3_conv2 = conv3x3(64, 64)
        self.layer3_bn2 = nn.BatchNorm2d(64)
        self.layer3_extra_conv1 = conv3x3(32, 64, stride=2)  # 调整residual的通道数
        self.layer3_extra_bn1 = nn.BatchNorm2d(64)
        self.layer3_extra_conv2 = conv3x3(64, 64)
        self.layer3_extra_bn2 = nn.BatchNorm2d(64)
        self.gelub31 = RayLU(trial, 10)
        self.gelub32 = RayLU(trial, 11)
        self.gelub33 = RayLU(trial, 12)
        self.gelub34 = RayLU(trial, 13)

        self.layer3_downsample = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )

        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)

        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)

        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


learning_rate = 0.001
num_epochs = 100


# 定义目标函数
def objective(trial):
    # 模型实例化
    model = ResNet(trial)
    model.to(device)
    curr_lr = learning_rate
    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器

    #history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

    for epoch in range(num_epochs):
        optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器
        train_loss, train_correct = 0.0, 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            train_correct += (predictions == labels).sum().item()

        # Decay learning rate
        if (epoch + 1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        valid_loss, valid_correct = 0.0, 0
        model.eval()
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            valid_correct += (predictions == labels).sum().item()
        ##不好的trial剪枝    
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        valid_loss = valid_loss / len(test_loader.sampler)
        valid_acc = valid_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} \t Train Loss:{:.4f} Valid Loss:{:.4f} \t Train Acc:{:.2f} %  Valid Acc:{:.2f} %".format(
            epoch + 1, num_epochs,
            train_loss,
            valid_loss,
            train_acc,
            valid_acc))
        # history['train_loss'].append(train_loss)
        # history['valid_loss'].append(valid_loss)
        # history['train_acc'].append(train_acc)
        # history['valid_acc'].append(valid_acc)

    return valid_acc


study = optuna.create_study(direction='maximize')

# 运行优化过程
study.optimize(objective, n_trials=20)

# 打印最佳参数和目标值
print('Best trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print('    {}: {}'.format(key, value))

Files already downloaded and verified


[I 2024-09-28 12:03:17,121] A new study created in memory with name: no-name-68596a98-ff02-40dd-bc78-4c85e467d3ac


Epoch:1/100 	 Train Loss:1.5924 Valid Loss:1.5206 	 Train Acc:40.88 %  Valid Acc:45.91 %
Epoch:2/100 	 Train Loss:1.2400 Valid Loss:1.1888 	 Train Acc:55.58 %  Valid Acc:57.44 %
Epoch:3/100 	 Train Loss:1.0427 Valid Loss:0.9666 	 Train Acc:62.93 %  Valid Acc:65.90 %
Epoch:4/100 	 Train Loss:0.9365 Valid Loss:0.8835 	 Train Acc:66.62 %  Valid Acc:68.39 %
Epoch:5/100 	 Train Loss:0.8655 Valid Loss:0.8234 	 Train Acc:69.26 %  Valid Acc:71.10 %
Epoch:6/100 	 Train Loss:0.8081 Valid Loss:0.7862 	 Train Acc:71.50 %  Valid Acc:72.32 %
Epoch:7/100 	 Train Loss:0.7565 Valid Loss:0.7661 	 Train Acc:73.47 %  Valid Acc:73.18 %
Epoch:8/100 	 Train Loss:0.7003 Valid Loss:0.6973 	 Train Acc:75.40 %  Valid Acc:75.80 %
Epoch:9/100 	 Train Loss:0.6593 Valid Loss:0.6970 	 Train Acc:76.60 %  Valid Acc:76.43 %
Epoch:10/100 	 Train Loss:0.6254 Valid Loss:0.6772 	 Train Acc:78.04 %  Valid Acc:77.05 %
Epoch:11/100 	 Train Loss:0.5883 Valid Loss:0.6461 	 Train Acc:79.24 %  Valid Acc:77.95 %
Epoch:12/100 	 Trai

[I 2024-09-28 13:23:25,538] Trial 0 finished with value: 86.76 and parameters: {'sigma_1': 3.423476052186064, 'sigma_2': 4.098131311369579, 'sigma_3': 1.3439461442440241, 'sigma_4': 2.592789434602616, 'sigma_5': 2.4784358311579053, 'sigma_6': 4.121806497231036, 'sigma_7': 2.3422659667291836, 'sigma_8': 4.977929158208426, 'sigma_9': 2.340357248697875, 'sigma_10': 2.008432233176517, 'sigma_11': 4.112695128215287, 'sigma_12': 4.541384343668526, 'sigma_13': 3.2551058020068773}. Best is trial 0 with value: 86.76.


Epoch:100/100 	 Train Loss:0.1571 Valid Loss:0.4801 	 Train Acc:94.44 %  Valid Acc:86.76 %
Epoch:1/100 	 Train Loss:1.4795 Valid Loss:1.6203 	 Train Acc:45.19 %  Valid Acc:42.00 %
Epoch:2/100 	 Train Loss:1.1960 Valid Loss:1.0485 	 Train Acc:57.00 %  Valid Acc:62.23 %
Epoch:3/100 	 Train Loss:1.0568 Valid Loss:1.0018 	 Train Acc:62.01 %  Valid Acc:64.50 %
Epoch:4/100 	 Train Loss:0.9667 Valid Loss:0.8914 	 Train Acc:65.35 %  Valid Acc:67.74 %
Epoch:5/100 	 Train Loss:0.8785 Valid Loss:0.8787 	 Train Acc:68.68 %  Valid Acc:68.55 %
Epoch:6/100 	 Train Loss:0.8111 Valid Loss:0.7748 	 Train Acc:71.33 %  Valid Acc:73.09 %
Epoch:7/100 	 Train Loss:0.7559 Valid Loss:0.7115 	 Train Acc:73.36 %  Valid Acc:74.81 %
Epoch:8/100 	 Train Loss:0.6977 Valid Loss:0.6798 	 Train Acc:75.56 %  Valid Acc:76.51 %
Epoch:9/100 	 Train Loss:0.6565 Valid Loss:0.6716 	 Train Acc:76.92 %  Valid Acc:76.97 %
Epoch:10/100 	 Train Loss:0.6248 Valid Loss:0.6360 	 Train Acc:78.09 %  Valid Acc:78.22 %
Epoch:11/100 	 Tra

...

Epoch:100/100 	 Train Loss:0.1446 Valid Loss:0.4766 	 Train Acc:94.80 %  Valid Acc:87.37 %
Epoch:1/100 	 Train Loss:1.4858 Valid Loss:1.2230 	 Train Acc:45.28 %  Valid Acc:55.80 %
Epoch:2/100 	 Train Loss:1.1330 Valid Loss:1.0171 	 Train Acc:59.07 %  Valid Acc:63.48 %
Epoch:3/100 	 Train Loss:0.9950 Valid Loss:0.9477 	 Train Acc:64.56 %  Valid Acc:66.22 %
Epoch:4/100 	 Train Loss:0.9072 Valid Loss:0.8739 	 Train Acc:67.76 %  Valid Acc:69.16 %
Epoch:5/100 	 Train Loss:0.8194 Valid Loss:0.7939 	 Train Acc:70.97 %  Valid Acc:71.21 %
Epoch:6/100 	 Train Loss:0.7620 Valid Loss:0.7307 	 Train Acc:73.01 %  Valid Acc:75.04 %
Epoch:7/100 	 Train Loss:0.7029 Valid Loss:0.7332 	 Train Acc:75.25 %  Valid Acc:74.49 %
Epoch:8/100 	 Train Loss:0.6553 Valid Loss:0.6371 	 Train Acc:76.91 %  Valid Acc:78.01 %
Epoch:9/100 	 Train Loss:0.6140 Valid Loss:0.6182 	 Train Acc:78.34 %  Valid Acc:78.87 %
Epoch:10/100 	 Train Loss:0.5752 Valid Loss:0.6459 	 Train Acc:79.94 %  Valid Acc:78.46 %
Epoch:11/100 	 Tra

[I 2024-09-29 13:24:24,837] Trial 18 finished with value: 87.33999999999999 and parameters: {'sigma_1': 2.7024356361231927, 'sigma_2': 3.9679345761297053, 'sigma_3': 3.4105596936675955, 'sigma_4': 1.6812633123351326, 'sigma_5': 0.7211859155353433, 'sigma_6': 3.3831713546077644, 'sigma_7': 3.331247144811644, 'sigma_8': 3.025150310748803, 'sigma_9': 3.1695391281493617, 'sigma_10': 0.654886961988781, 'sigma_11': 3.0469020180884554, 'sigma_12': 3.7067735456569433, 'sigma_13': 1.0558307429209117}. Best is trial 16 with value: 87.64.


Epoch:100/100 	 Train Loss:0.1464 Valid Loss:0.4666 	 Train Acc:94.76 %  Valid Acc:87.34 %
Epoch:1/100 	 Train Loss:1.5069 Valid Loss:1.3054 	 Train Acc:44.28 %  Valid Acc:52.95 %
Epoch:2/100 	 Train Loss:1.1344 Valid Loss:1.0242 	 Train Acc:59.01 %  Valid Acc:63.44 %
Epoch:3/100 	 Train Loss:0.9736 Valid Loss:0.9358 	 Train Acc:65.13 %  Valid Acc:67.18 %
Epoch:4/100 	 Train Loss:0.8719 Valid Loss:0.8092 	 Train Acc:68.87 %  Valid Acc:71.16 %
Epoch:5/100 	 Train Loss:0.7886 Valid Loss:0.7844 	 Train Acc:72.45 %  Valid Acc:72.69 %
Epoch:6/100 	 Train Loss:0.7258 Valid Loss:0.7279 	 Train Acc:74.44 %  Valid Acc:75.06 %
Epoch:7/100 	 Train Loss:0.6739 Valid Loss:0.6824 	 Train Acc:76.54 %  Valid Acc:77.05 %
Epoch:8/100 	 Train Loss:0.6317 Valid Loss:0.7362 	 Train Acc:78.09 %  Valid Acc:74.99 %
Epoch:9/100 	 Train Loss:0.5969 Valid Loss:0.6703 	 Train Acc:79.24 %  Valid Acc:76.94 %
Epoch:10/100 	 Train Loss:0.5688 Valid Loss:0.5644 	 Train Acc:80.20 %  Valid Acc:80.50 %
Epoch:11/100 	 Tra

[I 2024-09-29 14:23:13,059] Trial 19 finished with value: 87.11 and parameters: {'sigma_1': 2.372211917995976, 'sigma_2': 4.696423499963366, 'sigma_3': 4.328879601640268, 'sigma_4': 0.6471922939677988, 'sigma_5': 1.7514318969792253, 'sigma_6': 2.3662396501741343, 'sigma_7': 3.068965447439853, 'sigma_8': 2.094138834469828, 'sigma_9': 3.929064548616785, 'sigma_10': 1.2050553466740448, 'sigma_11': 1.7274097849914685, 'sigma_12': 4.890422164869193, 'sigma_13': 1.5680232782184458}. Best is trial 16 with value: 87.64.


Epoch:100/100 	 Train Loss:0.1481 Valid Loss:0.4838 	 Train Acc:94.79 %  Valid Acc:87.11 %
Best trial:
  Value:  87.64
  Params: 
    sigma_1: 2.552892614417707
    sigma_2: 3.4882592168857727
    sigma_3: 4.352113072004928
    sigma_4: 0.738816595610332
    sigma_5: 0.1056008808873683
    sigma_6: 3.587607017492251
    sigma_7: 3.4530867515762034
    sigma_8: 2.8790174876906804
    sigma_9: 3.1238297211006296
    sigma_10: 0.12856715281533626
    sigma_11: 3.217293494647598
    sigma_12: 2.7807219332711726
    sigma_13: 1.1450978029061871


In [1]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna  #载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False,
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)

Files already downloaded and verified


In [3]:
##看看其他激活函数，比如RayLU
class Raylu(nn.Module):  #带参数的gelu激活函数
    def __init__(self):
        super().__init__()
        self.sigma = nn.Parameter(2 * torch.randn(1))  #可学习参数 sigma

    def forward(self, input):
        x = torch.where(input >= 0, input, input * torch.exp(-input ** 2 / self.sigma ** 2 / 2))
        return x


# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        #self.relu = nn.ReLU()
        self.Raylu1 = Raylu()
        self.Raylu2 = Raylu()
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        #out = self.relu(out)
        out = self.Raylu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.Raylu2(out)
        return out


# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        #self.relu = nn.ReLU()
        self.Raylu3 = Raylu()
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        #out = self.relu(out)
        out = self.Raylu3(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


#model1 = ResidualBlock()
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate=0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#各个含有GELU的激活函数赋初值
z = model.Raylu3.sigma
nn.init.constant_(z, 6)
z101 = model.layer1[0].Raylu1.sigma
nn.init.constant_(z101, 4)
z102 = model.layer1[0].Raylu2.sigma
nn.init.constant_(z102, 4)
z111 = model.layer1[1].Raylu1.sigma
nn.init.constant_(z111, 3)
z112 = model.layer1[1].Raylu2.sigma
nn.init.constant_(z112, 3)
z201 = model.layer2[0].Raylu1.sigma
nn.init.constant_(z201, 5)
z202 = model.layer2[0].Raylu2.sigma
nn.init.constant_(z202, 5)
z211 = model.layer2[1].Raylu1.sigma
nn.init.constant_(z211, 5)
z212 = model.layer2[1].Raylu2.sigma
nn.init.constant_(z212, 5)
z301 = model.layer3[0].Raylu1.sigma
nn.init.constant_(z301, 2)
z302 = model.layer3[0].Raylu2.sigma
nn.init.constant_(z302, 4)
z311 = model.layer3[1].Raylu1.sigma
nn.init.constant_(z311, 2)
z312 = model.layer3[1].Raylu2.sigma
nn.init.constant_(z312, 3)


############################################

# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


# Train the model
total_step = len(train_loader)
curr_lr = learning_rate

historycifaRaylu = {'Raylu_train_loss': [], 'Raylu_valid_loss': [], 'Raylu_train_acc': [], 'Raylu_valid_acc': []}
num_epochs = 80

for epoch in range(num_epochs):
    Raylu_train_loss, Raylu_train_correct = 0.0, 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        Raylu_train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(outputs.data, 1)  #最大化输出得到标签0-9
        Raylu_train_correct += (predictions == labels).sum().item()  #对比标签得到正确的标签个数   

    # Decay learning rate
    if (epoch + 1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)
    Raylu_valid_loss, Raylu_val_correct = 0.0, 0
    model.eval()  #评估模式,非训练模式,batch normalization 和 dropout关闭
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        Raylu_valid_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(outputs.data, 1)  #最大化输出得到标签0-9
        Raylu_val_correct += (predictions == labels).sum().item()  #对比标签得到正确的标签个数

    Raylu_train_loss = Raylu_train_loss / len(train_loader.sampler)  #除以48000
    Raylu_train_acc = Raylu_train_correct / len(train_loader.sampler) * 100
    Raylu_valid_loss = Raylu_valid_loss / len(test_loader.sampler)  #除以12000
    Raylu_valid_acc = Raylu_val_correct / len(test_loader.sampler) * 100

    print("Epoch:{}/{} \t AVERAGE TL:{:.4f} AVERAGE VL:{:.4f} \t AVERAGE TA:{:.2f} %  AVERAGE VA:{:.2f} %".format(
        epoch + 1, num_epochs,
        Raylu_train_loss,
        Raylu_valid_loss,
        Raylu_train_acc,
        Raylu_valid_acc))
    historycifaRaylu['Raylu_train_loss'].append(Raylu_train_loss)  #每次训练和验证记录下来，后面画图   
    historycifaRaylu['Raylu_valid_loss'].append(Raylu_valid_loss)
    historycifaRaylu['Raylu_train_acc'].append(Raylu_train_acc)
    historycifaRaylu['Raylu_valid_acc'].append(Raylu_valid_acc) 

Epoch:1/80 	 AVERAGE TL:1.6360 AVERAGE VL:1.5498 	 AVERAGE TA:39.24 %  AVERAGE VA:44.04 %
Epoch:2/80 	 AVERAGE TL:1.2425 AVERAGE VL:1.0412 	 AVERAGE TA:55.47 %  AVERAGE VA:62.48 %
Epoch:3/80 	 AVERAGE TL:1.0140 AVERAGE VL:0.9309 	 AVERAGE TA:63.95 %  AVERAGE VA:66.75 %
Epoch:4/80 	 AVERAGE TL:0.8664 AVERAGE VL:0.8680 	 AVERAGE TA:69.17 %  AVERAGE VA:69.56 %
Epoch:5/80 	 AVERAGE TL:0.7861 AVERAGE VL:0.7686 	 AVERAGE TA:72.15 %  AVERAGE VA:73.09 %
Epoch:6/80 	 AVERAGE TL:0.7069 AVERAGE VL:0.7657 	 AVERAGE TA:75.13 %  AVERAGE VA:73.74 %
Epoch:7/80 	 AVERAGE TL:0.6527 AVERAGE VL:0.6688 	 AVERAGE TA:77.09 %  AVERAGE VA:76.68 %
Epoch:8/80 	 AVERAGE TL:0.5998 AVERAGE VL:0.6221 	 AVERAGE TA:79.08 %  AVERAGE VA:79.01 %
Epoch:9/80 	 AVERAGE TL:0.5619 AVERAGE VL:0.5747 	 AVERAGE TA:80.22 %  AVERAGE VA:80.03 %
Epoch:10/80 	 AVERAGE TL:0.5191 AVERAGE VL:0.5607 	 AVERAGE TA:81.96 %  AVERAGE VA:80.89 %
Epoch:11/80 	 AVERAGE TL:0.4952 AVERAGE VL:0.5721 	 AVERAGE TA:82.87 %  AVERAGE VA:80.85 %
Epoch:12